## Rede Neural

In [ ]:
from torch.utils.data import Dataset

In [ ]:
# init len e getitem são metodos especiais e sempre precisam existir

class Word2VecDataset(Dataset):
    def __init__(self, dataset):
        self.data = []

        for row in dataset:
            self.data.extend(row['window'])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
my_dataset = Word2VecDataset(dataset)

In [ ]:
len(my_dataset)

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(my_dataset, batch_size=1024, shuffle=True) 
# shuffle: ordem pode enviesar o treinamento
# batch_size: quanto mais exemplos passamos garante com que o treino seja menos caótico,
#  porque o erro vai ficar mais "smooth"
#  ao invés de atualizar a cada exemplo, atualizamos por lote
#  batch muito grande pode fazer estourar a memória da gpu (com datasets grandes)


In [ ]:
import torch
from torch import nn    

In [ ]:
class Word2Vec(nn.module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()

        # camadas da nn
        #   camada de embedding
        #   vocab: tamanho da entrada, mapea cada token único
        #   embedding: tamanho de saída
        #   tensor: array multidimensional necessário do pytorch. pra cada índice que eu tenho,
        #           retorna um vetor de tamanho = embedding_dim
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        #   fully connected
        #   entrada: tamanho do embedding
        #   saída: tamanho do vocabulário ([0.007, 0.003, 0.96, ..., 0.001, 0.0001, 0.001]),
        #        pegamos qual a palavra por ID / posição
        #        sempre normalizamos pra soma do vetor dar 1
        self.expand = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        # só precisamos fazer o forward, o backpropagation é gerenciado pela biblioteca
        embed_vector = self.embed(x)
        output = self.expand(embed_vector)

        return output
    

In [ ]:
embedding_test = nn.Embedding(len(vocab), 50)

display(embedding_test(torch.tensor(10))) # retorna um vetor de tamanho 50 inicializado de forma aleatória

display(embedding_test(torch.tensor(10)).shape) # 50

In [ ]:
network = Word2Vec(len(vocab), 50)

In [ ]:
network(torch.tensor(10)) # chama o forward

display(network(torch.tensor(10)).shape)

In [ ]:
result = network(torch.tensor(10))

display(result.argmax())

result[30983]

In [ ]:
id2token[10], id2token[30983]

In [ ]:
# ultimo passo

In [ ]:
LR = 0.01
EPOCHS = 5

loss_fn = nn.CrossEntropyLoss()
# se eu tenho valores que são iguais vai tender a 0. Quanto mais dif tende a infinito
optimizer = torch.optim.AdamW(network.parameters(), lr=LR)
# otimizador: baseados no gradiente descendente, mas tem alguns ajustes de LR de forma dinâmica
#   pra garantir aprendizado
#   no começo ajustamos bastante, talvez no final precisemos ajustar bem pouco
#   outro uso: você usou um tamanho de passo, se for bom aumenta, se começar a piorar diminui

# o dado e a rede precisa estar no mesmo device
device = torch.device('cuda')
network.to(device)


In [ ]:
from tqdm import tqdm

all_losses = []

for i in range(EPOCHS):
    epoch_loss = 0

    for input, context in dataloader:
        input = input.to(device)
        print('input: ', input)
        print('input shape: ', input.shape)
        context = context.to(device)
        print('context: ', context)
        print('context shape: ', context.shape)

        # input: primeiro elemento da tupla
        # context: segundo elemento da tupla
        # ('house', 'dog')


        output = network(input)

        optimizer.zero_grad()
        # fazemos pra fazer uma nova atualização de peso

        loss = loss_fn(output, context) # saber quanto erramoszes
        # debug
        break

        epoch_loss += loss.item() # monitorar as losszes

        loss.backward() # calcula os gradientes
        optimizer.step() # atualiza os pesos

    # torch.save(model.state_dict(), f'word2vec-{i}.pt')
    all_losses.append(epoch_loss)
    print(f"Epoch {i} loss: {epoch_loss}")

In [ ]:
from tqdm import tqdm

all_losses = []

for i in range(EPOCHS):
    epoch_loss = 0

    for input, context in tqdm(dataloader):
        input = input.to(device)
        context = context.to(device)

        output = network(input)

        optimizer.zero_grad()
        # fazemos pra fazer uma nova atualização de peso

        loss = loss_fn(output, context) # saber quanto erramoszes
        epoch_loss += loss.item() # monitorar as losszes

        loss.backward() # calcula os gradientes
        optimizer.step() # atualiza os pesos

    # torch.save(model.state_dict(), f'word2vec-{i}.pt')
    all_losses.append(epoch_loss)
    print(f"Epoch {i} loss: {epoch_loss}")

In [ ]:
torch.save(network.state_dict(), 'word2vec.pt')
torch.load('word2vec.pt')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(all_losses)

In [ ]:
# como acessar os embeddings?

word2vecs = network.expand.weight.cpu().detach().numpy()

In [ ]:
word2vecs.shape

In [ ]:
word2vecs[token2id['here']]

In [ ]:
# distancia das palavras

from scipy.spatial import distance

In [ ]:
w1 = word2vecs[token2id['father']]
w2 = word2vecs[token2id['mother']]

distance.pdist([w1, w2], 'cosine')

In [ ]:
w1 = word2vecs[token2id['mother']]
w2 = word2vecs[token2id['hate']]

distance.pdist([w1, w2], 'cosine')

In [ ]:
w1 = word2vecs[token2id['hate']]
w2 = word2vecs[token2id['speech']]

distance.pdist([w1, w2], 'cosine')

In [ ]:
w1 = word2vecs[token2id['mars']]
w2 = word2vecs[token2id['milk']]

distance.pdist([w1, w2], 'cosine')

In [ ]:
# calcular rankeamento de palavras mais similares
# pode ser feito com sklearn cosine similarity

word_embedding = word2vecs[token2id['hate']]

In [ ]:
# visualização dos embeddings, transformando os vetores em 2 dimensões    
# TAREFINHA DE CASAAAAAAAAAAAA